In [1]:
%load_ext autoreload

# --------------- #
# region: Imports #
import os
import sys
module_path = os.path.abspath('../../..')
if module_path not in sys.path:
  sys.path.insert(0, module_path)
# endregion   #
# --------------- #

from gem.primitives import Location, Vector

In [5]:
v = Vector(1, 0, direction=0)
print(v)
v.rotate(3)
print(v)

Vector(direction=0,forward=1,right=0,backward=0,left=0
Vector(direction=3,forward=0,right=1,backward=0,left=0
